In [101]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import pyarrow

In [9]:
#!pip install pyarrow

In [102]:
cwd = os.getcwd()
cwd

'/Users/michelle/MIDS/210 Capstone'

In [103]:
joined = pd.read_parquet('joined_datetime.parquet', engine='pyarrow')

income = pd.read_csv('income_data.csv', low_memory=False)


In [97]:
#joined.head()

In [98]:
#elevations.head()

In [104]:
def str_to_int(zip_code):
    zip_code = zip_code
    try:
        zip_code = int(zip_code)
    except:
        print(zip_code)
    return zip_code

In [99]:
#income[income['Zip Code']=='Unallocated']

# Dropping rows where income dataset doesn't have zip codes
Can't supplement; dropping

In [36]:
idx = income[ (income['Zip Code'] == 'Unallocated') 
                  | (income['Zip Code'] == 'Suppressed') 
                 | (income['Zip Code'] == 'Suppressed1')
                 | (income['Zip Code'] == 'Unallocated2')].index
income.drop(idx , inplace=True)

In [37]:
# checking
income[income['Zip Code']=='Unallocated']

,Taxable Year,Zip Code,State,City,County,Returns,CA AGI,Total Tax Liability,CountyLatitude,CountyLongitude,Geo County,Geo City,Geo ZipCode


In [38]:
# converting zip code column to ints
income['Zip Code'] = income['Zip Code'].apply(str_to_int)

In [53]:
# get unique zips from income dataset
unique_income_zips = set(income['Zip Code'].unique())
len(unique_income_zips)

2534

In [43]:
# converting schools joined dataset zip code column to ints
joined['school_zip'] = joined['school_zip'].apply(str_to_int)

In [54]:
# get unique zips from schools dataset
unique_school_zips = set(joined['school_zip'].unique())
len(unique_school_zips)

1539

In [58]:
print("{} unique income dataset zips".format(len(unique_income_zips)))
print("{} unique school zips".format(len(unique_school_zips)))

2534 unique income dataset zips
1539 unique school zips


In [59]:
# getting number of same unique zip codes across both datasets
intersect_zips = unique_income_zips.intersection(unique_school_zips)
len(intersect_zips)

1527

# Check each year of income dataset for # unique zip codes

In [64]:
income['Taxable Year'].value_counts().sort_index(ascending=True)

1992    2314
1993    2311
1994    2298
1995    2365
1996    2349
1997    2354
1998    2360
1999    2355
2000    2362
2001    2364
2002    2373
2003    2385
2004    2390
2005    2404
2006    2377
2007    2373
2008    2384
2009    2380
2010    2380
2011    2371
2012    2368
2013    2365
2014    2364
2015    2378
2016    2365
2017    2362
2018    2361
2019    2360
2020    2358
Name: Taxable Year, dtype: int64

In [70]:
years = list(income['Taxable Year'].unique())
len(years)

29

In [71]:
# make dict of unique zip codes per year
list_unique_zips_per_year = {}
for i in years:
    temp = income[income['Taxable Year'] == i]
    temp_unique_zips = set(temp['Zip Code'].unique())
    list_unique_zips_per_year[i] = temp_unique_zips

In [74]:
#list_unique_zips_per_year

In [72]:
len(list_unique_zips_per_year)

29

In [87]:
# function to get the count and list of intersecting zip codes between unique school zips
# and unique zips per year
def check_zip_intersect(unique_school_zips, year, unique_income_zips_year):
    unique_school_zips = unique_school_zips
    unique_income_zips_year = unique_income_zips_year
    year = year
    intersect_zips = unique_income_zips_year.intersection(unique_school_zips)
    #print("Number of intersecting zip codes for year {}:\n{}\n\n".format(year, len(intersect_zips)))
    return len(intersect_zips), intersect_zips

In [88]:
# dumping into dictionaries:
# key is year; value is the number of zip codes that intersect
year_and_count_intersect_zips = {}
# key is year; value is the list of zip codes in the intersect
year_and_list_intersect_zips = {}
for k, v in list_unique_zips_per_year.items():
    year_and_count_intersect_zips[k], year_and_list_intersect_zips[k] = check_zip_intersect(unique_school_zips, k, v)



In [91]:
year_and_count_intersect_zips

{2015: 1527,
 2003: 1518,
 2017: 1525,
 2004: 1520,
 2007: 1517,
 1995: 1488,
 2010: 1525,
 1992: 1446,
 2019: 1524,
 2001: 1511,
 2018: 1525,
 1998: 1501,
 2014: 1525,
 2016: 1524,
 2009: 1525,
 1994: 1459,
 2006: 1517,
 2012: 1524,
 2013: 1524,
 2020: 1525,
 2011: 1526,
 2002: 1513,
 2008: 1525,
 1996: 1490,
 2000: 1506,
 1999: 1503,
 1997: 1493,
 2005: 1523,
 1993: 1458}

In [96]:
# filter to years 2002-2017
filtered_counts = {key: year_and_count_intersect_zips[key] for key in range(2002,2018)}
filtered_counts

{2002: 1513,
 2003: 1518,
 2004: 1520,
 2005: 1523,
 2006: 1517,
 2007: 1517,
 2008: 1525,
 2009: 1525,
 2010: 1525,
 2011: 1526,
 2012: 1524,
 2013: 1524,
 2014: 1525,
 2015: 1527,
 2016: 1524,
 2017: 1525}

In [100]:
#year_and_list_intersect_zips